In [1]:
# 代码参考了 https://github.com/itsnamgyu/reasoning-teacher

# 环境准备

In [ ]:
%pip install transformers==4.28.1
%pip install torch==2.1.2

Note: you may need to restart the kernel to use updated packages.


In [9]:
# 导入必要包
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import os
import json
import datasets
from torch.utils.data import DataLoader

In [3]:
# 下载InstructGPT作为教师模型生成的数据
# https://github.com/gyfffffff/llm-deploy/releases/tag/cot_data
# 将下载的三个json文件放入同级目录data下

In [4]:
# 下载模型

# windows
%pip install -U "huggingface-hub[cli]"
!$env:HF_ENDPOINT = "https://hf-mirror.com"
!huggingface-cli download --resume-download openai-community/gpt2 --local-dir ../../models/GPT-2

# linux
%pip install -U "huggingface-hub[cli]"
!export HF_ENDPOINT=https://hf-mirror.com
!huggingface-cli download --resume-download openai-community/gpt2 --local-dir ../../models/GPT-2

^C
Note: you may need to restart the kernel to use updated packages.


�ļ�����Ŀ¼��������﷨����ȷ��


F:\llm-deploy\docs\chapter2\models\GPT-2


e:\Anaconda\envs\dl2\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Fetching 26 files: 100%|██████████| 26/26 [00:00<00:00, 308.12it/s]


Note: you may need to restart the kernel to use updated packages.


'export' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


F:\llm-deploy\docs\chapter2\models\GPT-2


e:\Anaconda\envs\dl2\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Fetching 26 files: 100%|██████████| 26/26 [00:00<00:00, 4682.75it/s]


In [18]:
# 定义接下来要用到的变量
model_name_or_path = "../../models/GPT-2"  # 替换成自己的模型路径
# model_type = "decoder"

# 加载分词器和模型
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)

# 定义数据路径和dataloader
train_data_path = "data/F_zs_cot_date_understanding_good_train.jsonl"
val_data_path = "data/F_zs_cot_date_understanding_good_val.jsonl"
test_data_path = "data/F_zs_cot_date_understanding_good_test.jsonl"

def load_finetune_data(path):
    if os.path.exists(path):
        with open(path) as f:
            data = json.load(f)
        print(f"data length: {len(data['data'])}")
        return data
    else:
        return None

def get_flat_data(data):
    input_format = "{sample[question]} ###"
    label_format = " {sample[reasoning_completion]} --> {sample[answer]}"
    # 也可以使用下面的格式
    # input_format = "Q: {sample[question]}\n\nA: Let's think step by step.\n\n"
    # label_format = " {sample[reasoning_completion]}\n\nTherefore the answer is {sample[answer]}"
    flat_data = {
        "input": [],
        "label": [],
    }

    for key, samples in data["data"].items():
        for sample in samples:
            flat_data["input"].append(input_format.format(sample=sample))
            flat_data["label"].append(label_format.format(sample=sample))
    return flat_data

model_type = "decoder"

    
def tokenize(example):
    tokenizer.pad_token = tokenizer.eos_token
    if model_type == "decoder":
        # Tokenize and apply left side padding manually

        # Tokenize in vanilla Python list form
        it = tokenizer(
            example["input"],
            max_length=512,
            truncation=True
        )
        iids = it["input_ids"]
        if "label" in example:
            lids = tokenizer(
                example["label"],
                max_length=512,
                truncation=True
            )["input_ids"]
        else:
            lids = [list() for _ in range(len(iids))]

        lengths = []
        input_ids = []
        attention_mask = []
        label_ids = []
        for iid, lid in zip(iids, lids):
            lengths.append(len(iid) + len(lid))
            input_ids.append(iid + lid)
            attention_mask.append([1] * (len(iid) + len(lid)))
            label_ids.append([-100] * len(iid) + lid)

        # Pad full sequences
        lengths = torch.tensor(lengths)
        pad_lengths = (lengths.max() - lengths).tolist()
        for i, l in enumerate(pad_lengths):
            # Apply left side padding
            # Why? https://github.com/huggingface/transformers/issues/3021#issuecomment-1231526631
            input_ids[i] = [tokenizer.pad_token_id] * l + input_ids[i]
            attention_mask[i] = [0] * l + attention_mask[i]
            label_ids[i] = [tokenizer.pad_token_id] * l + label_ids[i]
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "labels": torch.tensor(label_ids, dtype=torch.long),
        }
    else:
        raise NotImplementedError(model_type)

def dataloader(path):
    data = load_finetune_data(path)
    flat_data = get_flat_data(data)
    dataset = datasets.Dataset.from_dict(flat_data)
    dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset))
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return DataLoader(dataset, batch_size=8, shuffle=True)

train_dataloader = dataloader(train_data_path)
val_dataloader = dataloader(val_data_path)
test_dataloader = dataloader(test_data_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



data length: 315


Map:   0%|          | 0/343 [00:00<?, ? examples/s]

data length: 19


Map:   0%|          | 0/19 [00:00<?, ? examples/s]

data length: 28


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [19]:
def train_step(batch):
    kwargs = {
        "input_ids": batch["input_ids"].to(device),
        "attention_mask": batch["attention_mask"].to(device),
        "labels": batch["labels"].to(device),
    }
    res = model(**kwargs)["loss"]
    return res

def val():
    model.eval()
    with torch.no_grad():
        for step, batch in enumerate(val_dataloader):
            loss = train_step(batch)
            print("Val Step: {}, Loss: {}".format(step, loss.item()))

def train():
    model.to(device)
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    for i in range(10):
        for step, batch in enumerate(train_dataloader):
            loss = train_step(batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            optimizer.zero_grad()
            if step % 10 == 0:
                print("Step: {}, Loss: {}".format(step, loss.item()))
        val()
        torch.save(model.state_dict(), "outputs/model_epoch_{}.pt".format(i))
            
train()


Step: 0, Loss: 5.864531517028809
Step: 10, Loss: 3.1689412593841553


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\charo\AppData\Local\Temp\ipykernel_8108\3851600502.py:33 in <module>                    │
│                                                                                                  │
│   30 │   │   val()                                                                               │
│   31 │   │   torch.save(model.state_dict(), "outputs/model_epoch_{}.pt".format(i))               │
│   32                                                                                             │
│ ❱ 33 train()                                                                                     │
│   34                                                                                             │
│                                                                                                  │
│ C:\Users\charo\AppData\Local\Temp\ipykernel_8108\3851600502.py:24 in train                       │
│                                                                                                  │
│   21 │   for i in range(10):                                                                     │
│   22 │   │   for step, batch in enumerate(train_dataloader):                                     │
│   23 │   │   │   loss = train_step(batch)                                                        │
│ ❱ 24 │   │   │   loss.backward()                                                                 │
│   25 │   │   │   torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)                │
│   26 │   │   │   optimizer.step()                                                                │
│   27 │   │   │   optimizer.zero_grad()                                                           │
│                                                                                                  │
│ e:\anaconda\envs\dl2\lib\site-packages\torch\_tensor.py:492 in backward                          │
│                                                                                                  │
│    489 │   │   │   │   create_graph=create_graph,                                                │
│    490 │   │   │   │   inputs=inputs,                                                            │
│    491 │   │   │   )                                                                             │
│ ❱  492 │   │   torch.autograd.backward(                                                          │
│    493 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    494 │   │   )                                                                                 │
│    495                                                                                           │
│                                                                                                  │
│ e:\anaconda\envs\dl2\lib\site-packages\torch\autograd\__init__.py:251 in backward                │
│                                                                                                  │
│   248 │   # The reason we repeat the same comment below is that                                  │
│   249 │   # some Python versions print out the first line of a multi-line function               │
│   250 │   # calls in the traceback and some print out the last line                              │
│ ❱ 251 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   252 │   │   tensors,                                                                           │
│   253 │   │   grad_tensors_,                                                                     │
│   254 │   │   retain_graph,                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [12]:

def test(ckpt_path=""):
    if ckpt_path:
        model.load_state_dict(torch.load(ckpt_path))
    model.eval()
    with torch.no_grad():
        for step, batch in enumerate(test_dataloader):
            kwargs = {
                "input_ids": batch["input_ids"],
                "attention_mask": batch["attention_mask"],
                "labels": batch["labels"],
            }
            res = model(**kwargs)
            loss = res["loss"]
            print("Test Step: {}, Loss: {}".format(step, loss.item()))

test()

Test Step: 0, Loss: 2.2565512657165527
Test Step: 1, Loss: 2.2542436122894287
Test Step: 2, Loss: 1.8640873432159424
Test Step: 3, Loss: 2.0942330360412598


In [24]:
# 验证效果

# 训练后的模型
ckpt_path = "outputs/model_epoch_9.pt"
model_trained = GPT2LMHeadModel.from_pretrained(model_name_or_path)
model_trained.load_state_dict(torch.load(ckpt_path))
model_trained.eval()

# 训练前模型
model_name_or_path = "../../models/GPT-2"  # 替换成自己的模型路径
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
model.eval()

# 测试
for step, batch in enumerate(test_dataloader):
    batchsize = batch["input_ids"].shape[0]
    batch["labels"][batch["labels"]==-100] = tokenizer.eos_token_id
    for i in range(batchsize):
        kwargs = {
            "input_ids": batch["input_ids"][i:i+1],
            "attention_mask": batch["attention_mask"][i:i+1],
            "labels": batch["labels"][i:i+1],
        }
        input = tokenizer.decode(batch["input_ids"][i], skip_special_tokens=True)
        label = tokenizer.decode(batch["labels"][i], skip_special_tokens=True)
        print("***input***:\n", input)
        print("***label***:\n", label)

        res_trained = model_trained(**kwargs)
        res_untrained = model(**kwargs)

        # 解码输出
        logits = res_trained['logits'].cpu()
        predict = torch.argmax(res_trained['logits'], dim=-1)
        predict_str = tokenizer.decode(predict[0], skip_special_tokens=True)
        print("***trained predict***:\n", predict_str)

        logits = res_untrained['logits'].cpu()
        predict = torch.argmax(res_untrained['logits'], dim=-1)
        predict_str = tokenizer.decode(predict[0], skip_special_tokens=True)
        print("***untrained predict***:\n", predict_str)


***input***:
 On May 9th, 2017 Jane bought 40 eggs. She ate one per day. Today she ran out of eggs. What is the date one year ago from today in MM/DD/YYYY?
Which choice is true? Answer choices: (A) 06/18/2016, (B) 06/25/2016, (C) 06/22/2016, (D) 09/18/2016, (E) 06/18/2030, (F) 07/18/2015. ###  

May 9th, 2017 is 5/9/2017. 

If we go back one year from that, it would be 5/9/2016. 

But Jane ate one egg per day, so 40 days before 5/9/2016 would be 4/19/2016. 

Therefore, the answer is 4/19/2016, or (C). --> A
***label***:
   

May 9th, 2017 is 5/9/2017. 

If we go back one year from that, it would be 5/9/2016. 

But Jane ate one egg per day, so 40 days before 5/9/2016 would be 4/19/2016. 

Therefore, the answer is 4/19/2016, or (C). --> A
***trained predict***:
 
TheTheTheTheTheTheTheTheTheThe On May 9th the, is 5 people on The has 50 hour hour. The was ate.. information. The was new next of person?? Jane? her MMF/DDYYYY Jane
 page is it from
 1 are
1 A
 1029/2017 1 orA) 06/18/2016, orC)